## K-means Clustering
As discussed in the lecture, K-means works by iterating through two steps:

1. Assigning data points to closest center
2. Recomputing cluster centers based on current assignment
When the cluster assignments don't change, the algorithm has converged.

### Example
We will consider an example with 2D data generated from a 2 component Gaussian Mixture Model. It is often important to test algorithms with toy data to ensure our implementations are correct.
#### Generating the toy data
We will generate and plot 20 data points from a mixture of 2 Gaussians (10 data points from each component).

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#Set the random seed for reproducability
np.random.seed(123)

#Generate the data
X1 = np.random.multivariate_normal(np.array([1, 1]), np.diag([.1, .1]), 10)#10 points from the first GMM component
X2 = np.random.multivariate_normal(np.array([3,3 ]), np.diag([.1, .1]), 10)#10 points from the second GMM component
X = np.concatenate((X1, X2))

#Plot the data
plt.plot(X[:, 0], X[:, 1], 'bo', markersize=10)
plt.xlim([0, 5]);
plt.ylim([0, 5]);

We see that the two distinct clusters are clearly visible. If we run the K-mean algorithm on these data we can recover these clusters. We write a function that implements the two steps in the K-means algorithm. The inputs to this algorithm are the data and the current cluster centers. It returns a cluster assignment based on the input centers, the cost after the assignment step and new cluster centers based on the new assignment. 

In [ ]:
def kmeans(data, centers):

    new_cent = np.zeros(centers.shape)
    #assignment step
    dist = np.zeros((data.shape[0], centers.shape[0]))
    for i in range(centers.shape[0]):
        dist[:,i] = np.sum((data - centers[i,:])**2, 1)

    assign = np.argmin(dist,1)

    #compute new centers and cost
    cost = 0
    for i in range(centers.shape[0]):
        new_cent[i,:] = np.mean(data[assign==i,:], 0)
        cost += np.sum(np.sum((data[assign==i,:] - centers[i,:])**2, 1))

    return new_cent, cost, assign

Let us run through one iteration of the K-means algorithm.

In [ ]:
#We need a value of K
K = 2 # we assume there are two clusters

#random starting points
centers = np.random.multivariate_normal(np.array([2, 2]), np.diag([1, 1]), K)

#run one iteration of K-means
new_cent, cost, assign = kmeans(X, centers)

#plot the assignment based on input cluster centers
col=['r', 'g'] #colour code for the two clusters
plt.plot(X[assign==0,0], X[assign==0,1], col[0]+'o', markersize=10)
plt.plot(X[assign==1,0], X[assign==1,1], col[1]+'o', markersize=10)
plt.plot(centers[0,0], centers[0,1], col[0]+'x', markersize=10, mew=2) #input center
plt.plot(centers[1,0], centers[1,1], col[1]+'x', markersize=10,mew=2) #input center
plt.xlim([0,5]);
plt.ylim([0,5]);

In [ ]:
#Plot the new centers
centers = new_cent # this will be the centers in the next iteration
plt.plot(X[assign==0,0], X[assign==0,1], col[0]+'o', markersize=10)
plt.plot(X[assign==1,0], X[assign==1,1], col[1]+'o', markersize=10)
plt.plot(centers[0,0], centers[0,1], col[0]+'x', markersize=10, mew=2)
plt.plot(centers[1,0], centers[1,1], col[1]+'x', markersize=10, mew=2) 
plt.xlim([0,5]);
plt.ylim([0,5]);

Write a for loop that calls the function kmeans for a number of iterations (say 4) and monitor any change in cluster assignment.

In [ ]:
### Your code

Write a while loop that allows you to terminate the algorithm when the cluster assignment doesn't change.

In [ ]:
### Your code

### K-means for Image compression (Vector Quantization)

The K-means algorithm can be used for image compression. In this example we will demonstrate it on an image of Mzee Jomo Kenyatta. We divide the image into 2-by-2 blocks which we treat as vectors in $\mathbf{R}^4$. We then cluster these blocks and use the assigned cluster centers to represent the image.

We will use the Python Imaging Library (PIL) to access the image. We will also use the scikit-learn toolbox of implementations of machine learning algorithms for K-means.

In [ ]:
import PIL
from sklearn.cluster import KMeans
import matplotlib.cm as cm # colour map to display grayscale image

#Obtain the image
img = PIL.Image.open('ADFL.JPG')

# Display the image
plt.imshow(np.asarray(img), cmap = cm.Greys_r)

In [ ]:
A = np.asarray(img)
print(A.shape)

We need to get the 2-by-2 blocks from the image array and flatten them to vectors in $\mathbf{R}^4$ before running K means.

In [ ]:
#get the vectors
px = img.size[0]
vq = np.zeros((int((px * px)/4), 4))
k = 0
for i in range(0, A.shape[0], 2):
    for j in range(0, A.shape[1], 2):
        vq[k,:] = np.ndarray.flatten(A[i:i+2, j:j+2])
        k += 1

kmeans = KMeans(init='k-means++', n_clusters=3, n_init=10)
kmeans.fit(vq)
cent = kmeans.cluster_centers_

Plot the quantized image

In [ ]:
#quantized image
img_mtx2 = np.zeros(img.size)
k = 0
for i in range(0, img.size[0], 2):
    for j in range(0, img.size[1], 2):
        img_mtx2[i:i+2, j:j+2] = np.reshape(np.round(cent[kmeans.predict(vq[k,:].reshape(1, -1))]),(2,2))
        k+=1
plt.imshow(img_mtx2, cmap = cm.Greys_r)